# SVM

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
from cvxopt import matrix
from cvxopt import solvers
import math
from scipy.spatial.distance import cdist
import time
from tqdm import tqdm
import pickle
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
import seaborn as sn

In [2]:
def ReadInput(c1,c2):
    df = pd.read_csv('./fashion_mnist/train.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==c1 ,data[:,784]==c2)
    X_train = data[index][:,:-1]
    Y_train = np.array([1 if i==c1 else -1 for i in data[index][:,-1]]).reshape((-1,1))


    df = pd.read_csv('./fashion_mnist/test.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==c1 ,data[:,784]==c2)
    X_test = data[index][:,:-1]
    Y_test = np.array([1 if i==c1 else -1 for i in data[index][:,-1]]).reshape((-1,1))


    df = pd.read_csv('./fashion_mnist/val.csv',header=None)
    data = df.to_numpy()
    index = np.logical_or(data[:,784]==c1 ,data[:,784]==c2)
    X_val = data[index][:,:-1]
    Y_val = np.array([1 if i==c1 else -1 for i in data[index][:,-1]]).reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, X_val, Y_val


<font color=red>DATA INPUT</font>

In [3]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = ReadInput(4,5)

<font color = 'red' size = 6cm>Binary Classification</font>

# PART 2 - 1.a 

In [4]:
def LinearSVM_fit(X,Y):
#     X = X_train/255
#     Y = Y_train
    G1 = np.diag(np.array([1]*Y.shape[0]))
    G2 = np.diag(np.array([-1]*Y.shape[0]))
    G = matrix(np.append(G1,G2,axis=0),tc='d')
    c = 1
    H = matrix(np.append(np.array([c]*Y.shape[0]),np.array([0]*Y.shape[0]),axis = 0),tc='d')
    P = matrix(np.dot(X,X.T)*np.dot(Y,Y.T),tc='d')
    Q = matrix(np.array([-1]*Y.shape[0]).T,tc='d')
    A = matrix(Y.T,tc='d')
    B = matrix(np.array(0).reshape((1,1)),tc='d')

    sol = solvers.qp(P,Q,G,H,A,B)

    alpha=np.array(sol['x'])

    #calculating the support vectors given a threshold
    threshold = 1e-10
    data = np.append(X,Y,axis = 1)
    data_with_alpha = np.append(data,alpha,axis = 1)
    
    supportVectors = data_with_alpha[np.where(data_with_alpha[:,-1] >= threshold)]
    print("Total supports vectors")
    print(supportVectors.shape)
    print('\n')
    # w = SUM( x*y*alpha )
    w = np.sum(supportVectors[:,:-2]*supportVectors[:,-2].reshape((-1,1))*supportVectors[:,-1].reshape((-1,1)),axis=0).reshape((-1,1))

#     pos_X = supportVectors[np.where(supportVectors[:,-2]==1)][:,:-2]
#     neg_X = supportVectors[np.where(supportVectors[:,-2]==-1)][:,:-2]
    XY = np.append(X,Y,axis=1)
    pos_X = XY[np.where(XY[:,-1]==1)][:,:-1]
    neg_X = XY[np.where(XY[:,-1]==-1)][:,:-1]
                
    print("aplha MIN ",alpha.max())
    print("aplha MAX ",alpha.min())
    b = -(np.dot(pos_X,w).min() + np.dot(neg_X,w).max())/2
    print(b)
    return w,b,supportVectors

In [5]:
def LinearSVM_predict(X,Y,w,b):
    pred = []
    for x in X:
        if (np.dot(w.T,x)+b) > 0:
            pred.append(1)
        else:
            pred.append(-1)

    count =0

    for i in range(len(pred)):
        if pred[i] == Y[i]:
            count+=1
    
    print("Accurarcy is ",count*100/Y.shape[0]," %") 

<font color=red>LEARNING</font>

In [36]:
lk = time.time()
w,b,supportVectors = LinearSVM_fit(X_train/255,Y_train)
print('total time for linear kernel SVM :', time.time()-lk)

     pcost       dcost       gap    pres   dres
 0: -1.8144e+02 -7.2340e+03  4e+04  2e+00  1e-12
 1: -1.0203e+02 -3.4492e+03  6e+03  3e-01  1e-12
 2: -3.3534e+01 -9.1974e+02  2e+03  7e-02  6e-13
 3: -1.1925e+01 -4.1859e+02  7e+02  2e-02  3e-13
 4: -3.6847e+00 -1.7770e+02  3e+02  8e-03  1e-13
 5: -1.2594e+00 -5.9907e+01  9e+01  2e-03  5e-14
 6: -6.7211e-01 -2.5705e+01  4e+01  9e-04  3e-14
 7: -4.1327e-01 -1.1925e+01  2e+01  3e-04  2e-14
 8: -5.1263e-01 -5.5495e+00  6e+00  1e-04  1e-14
 9: -5.7700e-01 -3.1645e+00  3e+00  4e-05  1e-14
10: -7.0634e-01 -2.0084e+00  1e+00  3e-06  2e-14
11: -9.3593e-01 -1.4177e+00  5e-01  7e-07  2e-14
12: -1.0291e+00 -1.2146e+00  2e-01  4e-16  2e-14
13: -1.0863e+00 -1.1319e+00  5e-02  4e-16  2e-14
14: -1.1048e+00 -1.1100e+00  5e-03  5e-16  2e-14
15: -1.1073e+00 -1.1074e+00  1e-04  5e-16  2e-14
16: -1.1073e+00 -1.1073e+00  1e-06  6e-16  2e-14
17: -1.1073e+00 -1.1073e+00  1e-08  5e-16  2e-14
Optimal solution found.
Total supports vectors
(80, 786)


aplha MIN  

<font color=red>PREDICTING</font>

In [24]:
LinearSVM_predict(X_test/255,Y_test,w,b)
LinearSVM_predict(X_val/255,Y_val,w,b)
LinearSVM_predict(X_train/255,Y_train,w,b)

Accurarcy is  99.8  %
Accurarcy is  99.6  %
Accurarcy is  100.0  %


# PART 2 - 1.b

In [25]:
def GaussianKernel(x,z,gamma):
    L2norm= cdist(x,z,'euclidean')
    return np.exp(-(L2norm**2)*gamma)


In [26]:
from tqdm import tqdm
def GausianSVM_fit(X,Y):
    G1 = np.diag(np.array([1]*Y.shape[0]))
    G2 = np.diag(np.array([-1]*Y.shape[0]))
    G = matrix(np.append(G1,G2,axis=0),tc='d')
    c = 1
    H = matrix(np.append(np.array([c]*Y.shape[0]),np.array([0]*Y.shape[0]),axis = 0),tc='d')
    Q = matrix(np.array([-1]*Y.shape[0]).T,tc='d')
    A = matrix(Y.T,tc='d')
    B = matrix(np.array(0).reshape((1,1)),tc='d')
    IP = []
    gamma = 0.05
    
    IP = GaussianKernel(X,X,gamma)

    P = matrix(IP*np.dot(Y,Y.T),tc='d')
    
    #-----------------------------------
    sol = solvers.qp(P,Q,G,H,A,B)
    alpha=np.array(sol['x'])
    #-----------------------------------
    
    print("aplha MIN ",alpha.max())
    print("aplha MAX ",alpha.min())
    
    #calculating the support vectors given a threshold
    threshold = 1e-5
    data = np.append(X,Y,axis = 1)
    data_with_alpha = np.append(data,alpha,axis = 1)

    supportVectors = data_with_alpha[np.where(data_with_alpha[:,-1] >= threshold)]
    print("Total supports vectors")
    print(supportVectors.shape)
    print('\n')

    XY = np.append(X,Y,axis=1)
    pos_X = XY[np.where(XY[:,-1]==1)][:,:-1]
    neg_X = XY[np.where(XY[:,-1]==-1)][:,:-1]

    #b
    y_i = supportVectors[:,-2]
    aplha_i = supportVectors[:,-1]
    b1 = np.dot(GaussianKernel(supportVectors[:,:-2],pos_X,gamma).T,(aplha_i*y_i)).min()
    
    b2 = np.dot(GaussianKernel(supportVectors[:,:-2],neg_X,gamma).T,(aplha_i*y_i)).max()



    b = -(b1+b2)/2
    return b,supportVectors

In [27]:
def GausianSVM_predict(X,Y,b,supportVectors):
    gamma =0.05
    
    y_i = supportVectors[:,-2]
    alpha_i =supportVectors[:,-1]
    

    wx = np.dot(GaussianKernel(supportVectors[:,:-2], X, gamma).T, (alpha_i*y_i)) 

    
    prediction = wx+b

    count = 0
    for i in range(X.shape[0]):
        if (prediction[i] >0 and Y[i]==1) or (prediction[i] <=0 and Y[i]== -1):
            count+=1
    
    print((count*100)/X.shape[0])

In [28]:
s =time.time()
b,supportVectors = GausianSVM_fit(X_train/255,Y_train)
print(b)
print(time.time()-s)

     pcost       dcost       gap    pres   dres
 0: -1.5598e+02 -6.9927e+03  3e+04  2e+00  1e-15
 1: -8.8657e+01 -3.4857e+03  6e+03  2e-01  1e-15
 2: -5.9446e+01 -7.3609e+02  8e+02  2e-02  2e-15
 3: -1.0523e+02 -2.6151e+02  2e+02  4e-03  2e-15
 4: -1.2171e+02 -1.8069e+02  6e+01  1e-03  1e-15
 5: -1.2850e+02 -1.5673e+02  3e+01  8e-15  1e-15
 6: -1.3314e+02 -1.4204e+02  9e+00  1e-14  1e-15
 7: -1.3446e+02 -1.3849e+02  4e+00  6e-15  1e-15
 8: -1.3538e+02 -1.3651e+02  1e+00  3e-15  1e-15
 9: -1.3573e+02 -1.3581e+02  8e-02  1e-14  1e-15
10: -1.3576e+02 -1.3576e+02  2e-03  2e-14  1e-15
11: -1.3576e+02 -1.3576e+02  6e-05  3e-16  1e-15
Optimal solution found.
aplha MIN  0.9999999936128763
aplha MAX  2.7568175816981765e-09
Total supports vectors
(1008, 786)


91.32317876815796


In [30]:
GausianSVM_predict(X_test/255,Y_test,b,supportVectors)


98.8


In [31]:
GausianSVM_predict(X_val/255,Y_val,b,supportVectors)

99.6


In [32]:
GausianSVM_predict(X_train/255,Y_train,b,supportVectors)

100.0


# PART 2 - 1.c

In [46]:

def SKlearn_LinearSVC(X_train, Y_train, X_test, Y_test, X_val, Y_val):
    clf = SVC(random_state=0, tol=1e-5,kernel ='linear')
    clf.fit(X_train/255,Y_train)
    print("Intercept ",clf.intercept_)
    print("Number of support Vectors ",clf.n_support_)

    Y_pred_test=clf.predict(X_test/255)
    Acc_test = accuracy_score(Y_test, Y_pred_test.ravel())*100
    print("Accuracy over TEST data : ",Acc_test)

    Y_pred_val=clf.predict(X_val/255)
    Acc_val = accuracy_score(Y_val, Y_pred_val.ravel())*100
    print("Accuracy over VALIDATION data : ",Acc_val)

    
        
    Y_pred_train=clf.predict(X_train/255)
    Acc_train = accuracy_score(Y_train, Y_pred_train.ravel())*100
    print("Accuracy over TRAIN data : ",Acc_train)
    
    
    
def SKlearn_SVC(X_train, Y_train, X_test, Y_test, X_val, Y_val):
    clf = SVC(gamma=0.05,kernel='rbf')
    clf.fit(X_train/255,Y_train)
    print("Intercept ",clf.intercept_)
    print("Number of support Vectors ",clf.n_support_)
    
    Y_pred_test=clf.predict(X_test/255)
    Acc_test = accuracy_score(Y_test, Y_pred_test.ravel())*100
    print("Accuracy over TEST data : ",Acc_test)

    Y_pred_val=clf.predict(X_val/255)
    Acc_val = accuracy_score(Y_val, Y_pred_val.ravel())*100
    print("Accuracy over VALIDATION data : ",Acc_val)

    
        
    Y_pred_train=clf.predict(X_train/255)
    Acc_train = accuracy_score(Y_train, Y_pred_train.ravel())*100
    print("Accuracy over TRAIN data : ",Acc_train)
    


In [47]:
SKlearn_LinearSVC(X_train, Y_train, X_test, Y_test, X_val, Y_val)

/home/vivek/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Intercept  [-0.49688327]
Number of support Vectors  [57 16]
Accuracy over TEST data :  99.8
Accuracy over VALIDATION data :  99.6
Accuracy over TRAIN data :  100.0


In [48]:
SKlearn_SVC(X_train, Y_train, X_test, Y_test, X_val, Y_val)

/home/vivek/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Intercept  [-0.32537323]
Number of support Vectors  [594 387]
Accuracy over TEST data :  99.6
Accuracy over VALIDATION data :  99.6
Accuracy over TRAIN data :  100.0


<font color = 'red' size = 6cm>Multi-Class Classification</font>

# PART 2 - 2.a

In [51]:
def multiClass_fit():
    for i in range(10):
        for j in tqdm(range(i+1,10)):
            X_train, Y_train, X_test, Y_test, X_val, Y_val = ReadInput(i,j)
            b,supportVectors = GausianSVM_fit(X_train/255,Y_train)
            model = [b,supportVectors]
            version = "model"+str(i)+str(j)
            with open(version,'wb') as f:
                pickle.dump(model,f)
        
        

In [52]:
def MultiClass_predict(X,Y):
    m = X.shape[0]
    gamma= 0.05
    count_matrix = np.zeros((m,10))
    score_matrix = np.zeros((m,10))

    for i in tqdm(range(10)):
        for j in range(i+1,10):
            modelNo = "model"+str(i)+str(j)
            with open('Models/'+modelNo,'rb') as f:
                model = pickle.load(f)
            b = model[0]
            supportVectors = model[1]

            y_i = supportVectors[:,-2]
            alpha_i =supportVectors[:,-1]

            wx = np.dot(GaussianKernel(supportVectors[:,:-2], X, gamma).T, (alpha_i*y_i)) 

            prediction_score = wx+b

            for k in range(X.shape[0]):

                if prediction_score[k] > 0:
                    score_matrix[k,i] += abs((prediction_score[k]))
                    count_matrix[k,i]+=1
                else:
                    score_matrix[k,j] += abs((prediction_score[k]))
                    count_matrix[k,j]+=1


    final_pred = []
    correct_pred_count = 0
    confusion_matrix = np.zeros((10,10))
    for i in tqdm(range(X.shape[0])):
        class_index = []
        max_value = count_matrix[i,:].max()
        for j in range(10):
            if max_value == count_matrix[i,j]:
                class_index.append(j)
        if len(class_index)==1:
            final_pred.append(class_index[0])
            if final_pred[i] == Y[i]:
                correct_pred_count+=1
                confusion_matrix[int(Y[i]),int(Y[i])]+=1
            else:
                confusion_matrix[int(final_pred[i]),int(Y[i])]+=1
                
        else:
            score_value = -math.inf
            for k in class_index:
                if score_matrix[i,k] > score_value:
                    score_value = score_matrix[i,k]
                    index = k
            final_pred.append(index)
            if final_pred[i] == Y[i]:
                correct_pred_count+=1
                confusion_matrix[int(Y[i]),int(Y[i])]+=1
            else:
                confusion_matrix[int(final_pred[i]),int(Y[i])]+=1
                
    print("\n")
    print("accuracy is ",(correct_pred_count*100)/Y.shape[0])
    return confusion_matrix

In [53]:
df = pd.read_csv('fashion_mnist/test.csv',header=None)
data = df.to_numpy()
X_test = data[:,:-1]
Y_test = data[:,-1]
X_test =X_test/255
confusion_matrix_2a_test = MultiClass_predict(X_test,Y_test)

100%|██████████| 5000/5000 [00:00<00:00, 52177.44it/s]



accuracy is  85.08


In [57]:
df = pd.read_csv('fashion_mnist/val.csv',header=None)
data = df.to_numpy()
X_val = data[:,:-1]
Y_val = data[:,-1]
X_val =X_val/255
confusion_matrix_2a_val = MultiClass_predict(X_val,Y_val)

100%|██████████| 2500/2500 [00:00<00:00, 36210.49it/s]



accuracy is  84.96


# PART 2 - 2.b

<font size = 5cm color = red>Multi-class SVM: Using the Scikit SVM </font>

In [59]:
def SKlearnOVO_fit(X,Y):
    
    for i in range(10):
        for j in range(i+1,10):
            model = SVC(gamma=0.05,kernel='rbf')
            model.fit(X,Y)
            Y_pred = model.predict(X)
            score = model.decision_function(X)
            model_parameters = [Y_pred, score]
            version = "SKlearn_Model"+str(i)+str(j)
            with open() as f:
                pickle.dump(model_parameters,f)
                
        
        
        

In [60]:
def SKlearn_MultiClass_predict(X,Y):
    m = X.shape[0]
    gamma= 0.05
    count_matrix = np.zeros((m,10))
    score_matrix = np.zeros((m,10))

    for i in range(10):
        for j in tqdm(range(i+1,10)):
            modelNo = "SKlearn_Model"+str(i)+str(j)
            with open('Models/'+modelNo,'rb') as f:
                model = pickle.load(f)
            
            prediction_score = model.decision_function(X)
            

            for k in range(X.shape[0]):

                if prediction_score[k] > 0:
                    score_matrix[k,i] += abs((prediction_score[k]))
                    count_matrix[k,i]+=1
                else:
                    score_matrix[k,j] += abs((prediction_score[k]))
                    count_matrix[k,j]+=1



    final_pred = []
    correct_pred_count = 0
    confusion_matrix = np.zeros((10,10))
    for i in tqdm(range(X.shape[0])):
        class_index = []
        max_value = count_matrix[i,:].max()
        for j in range(10):
            if max_value == count_matrix[i,j]:
                class_index.append(j)
        if len(class_index)==1:
            final_pred.append(class_index[0])
            if final_pred[i] == Y[i]:
                correct_pred_count+=1
                confusion_matrix[int(Y[i]),int(Y[i])]+=1
            else:
                confusion_matrix[int(final_pred[i]),int(Y[i])]+=1
        else:
            score_value = -math.inf
            for k in class_index:
                if score_matrix[i,k] > score_value:
                    score_value = score_matrix[i,k]
                    index = k
            final_pred.append(index)
            if final_pred[i] == Y[i]:
                correct_pred_count+=1
                confusion_matrix[int(Y[i]),int(Y[i])]+=1
            else:
                confusion_matrix[int(final_pred[i]),int(Y[i])]+=1
    
    print("\n")
    print("accuracy is ",(correct_pred_count*100)/Y.shape[0])
    return confusion_matrix
                

In [61]:
df = pd.read_csv('fashion_mnist/test.csv',header=None)
data = df.to_numpy()
X_test = data[:,:-1]
Y_test = data[:,-1].astype(int)
confusion_matrix_2b_test = SKlearn_MultiClass_predict(X_test/255,Y_test)

100%|██████████| 1/1 [00:07<00:00,  7.96s/it]
0it [00:00, ?it/s]
100%|██████████| 5000/5000 [00:00<00:00, 75658.11it/s]



accuracy is  88.08


In [63]:
df = pd.read_csv('fashion_mnist/val.csv',header=None)
data = df.to_numpy()
X_val = data[:,:-1]
Y_val = data[:,-1].astype(int)
confusion_matrix_2b_val = SKlearn_MultiClass_predict(X_val/255,Y_val)

100%|██████████| 1/1 [00:03<00:00,  3.84s/it]
0it [00:00, ?it/s]
100%|██████████| 2500/2500 [00:00<00:00, 65543.78it/s]



accuracy is  87.88


# PART 2 - 2.c

<font color =red size =5cm>Confusion Matrix</font>

In [55]:
%matplotlib qt
def ConfusionMatrix(confusionMatrix,title):
    df_cm = pd.DataFrame(confusionMatrix, index = [i for i in range(10)],
                      columns = [i for i in range(10)], dtype=int)
    plt.figure(figsize = (10,8))
    sn.heatmap(df_cm, annot=True,fmt='d')
    plt.xlabel("ACTUAL VALUE")
    plt.ylabel("PREDICTED VALUE")
    plt.title(title,color = 'red')
    plt.show()

In [56]:
ConfusionMatrix(confusion_matrix_2a_test,"Gaussian Kernel: Test")

In [58]:
ConfusionMatrix(confusion_matrix_2a_val,"Gaussian Kernel: Val")

In [62]:
ConfusionMatrix(confusion_matrix_2b_test,"Scikit SVM Kernel: Test")

In [64]:
ConfusionMatrix(confusion_matrix_2b_val,"Scikit SVM Kernel: Val")

# PART 2 - 2.d

<font color =red size =5cm>5-fold cross validation </font>

In [ ]:
df = pd.read_csv('./fashion_mnist/train.csv',header=None)
data = df.to_numpy()
X_train = data[:,:-1]/255
Y_train = data[:,-1].astype(int)

df = pd.read_csv('./fashion_mnist/test.csv',header=None)
data = df.to_numpy()
X_test = data[:,:-1]/255
Y_test = data[:,-1].astype(int)

In [20]:
C_parameter = [1e-5,1e-3,1,5,10]
# k_fold = KFold(n_splits=5)
sfK_fold=StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [17]:
meanscoresList = []
scoresMatrix = []
for c in tqdm(C_parameter):
    SVCmodel = SVC(C=c, kernel='rbf',gamma=0.05)
    scores = cross_val_score(SVCmodel, X_train, Y_train, cv=sfK_fold, n_jobs=-1)
    scoresMatrix.append(scores)
    meanScore = np.mean(scores)
    meanscoresList.append(meanScore)

__ [0.5664444444444444,
 0.5664444444444444,
 0.8787111111111111,
 0.8844,
 0.8842666666666666] __

In [ ]:
from joblib import Parallel,delayed
def svcParameter(c):
    SVCmodel = SVC(C=c, kernel='rbf',gamma=0.05,decision_function_shape='ovo')
    SVCmodel.fit(X_train,Y_train)
    Y_pred_test = SVCmodel.predict(X_test)
    Acc_test = accuracy_score(Y_test, Y_pred_test)*100
    print("Accuracy over TEST data : ",Acc_test)
    return Acc_test

 

acc_val = Parallel(n_jobs=5)(delayed(svcParameter)(i) for i in C_parameter )

# acc_val = [57.36, 57.36, 88.08, 88.28, 88.24]  20 MINUTES

In [25]:
Model_score = np.array([0.5664444444444444, 0.5664444444444444, 0.8787111111111111, 0.8844, 0.8842666666666666])*100
Acc_val = np.array([57.36, 57.36, 88.08, 88.28, 88.24] )
%matplotlib qt
x_axis = np.log10([1e-5, 1e-3, 1, 5, 10])
fig, ax = plt.subplots()
ax.set_ylabel('Scores %')
ax.set_xlabel('Differnt C-parameter Values')
ax.set_title('K-fold Cross_validation')
# ax.set_xticklabels(x_axis)
plt.xticks(x_axis)
plt.plot(x_axis,Model_score,label='Train scores', marker='o')
plt.plot(x_axis,Acc_val,label ='Test_accuracy', marker='x')
plt.legend()
plt.show()
